In [9]:
#!pip install stable-baselines3[extra]
# !conda install swig # needed to build Box2D in the pip install
# !pip install box2d-py # a repackaged version of pybox2d

In [1]:
import gym
import numpy as np
from stable_baselines3 import DQN,A2C
from stable_baselines3.common.evaluation import evaluate_policy

In [3]:
# Create environment
env = gym.make('LunarLander-v2')
print(env.action_space.sample())
print(env.observation_space.sample())

2
[-2.4402425  -1.2959294  -0.9281709  -0.35414     0.5344206  -0.89617175
  0.2890721  -0.537578  ]


In [4]:
# Instantiate the agent
model = A2C('MlpPolicy', env,
            verbose=1,tensorboard_log='./logs',
            learning_rate=5e-4,policy_kwargs={
                'net_arch':[256,256]
            })

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [5]:
# Train the agent
model.learn(total_timesteps=int(1e6),tb_log_name='A2C')
# Save the agent
model.save("dqn_a2c_net256_lunar")
del model  # delete trained model to demonstrate loading

Logging to ./logs\A2C_1
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 68.6     |
|    ep_rew_mean        | -473     |
| time/                 |          |
|    fps                | 222      |
|    iterations         | 100      |
|    time_elapsed       | 2        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -0.238   |
|    explained_variance | -0.0691  |
|    learning_rate      | 0.0005   |
|    n_updates          | 99       |
|    policy_loss        | -0.57    |
|    value_loss         | 165      |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 63.8     |
|    ep_rew_mean        | -493     |
| time/                 |          |
|    fps                | 247      |
|    iterations         | 200      |
|    time_elapsed       | 4        |
|    total_timesteps    | 1000     |
| train/      

In [14]:
# Load the trained agent
# NOTE: if you have loading issue, you can pass `print_system_info=True`
# to compare the system on which the model was trained vs the current one
# model = DQN.load("dqn_lunar", env=env, print_system_info=True)
model = DQN.load("dqn_a2c_net256_lunar", env=env)

# Evaluate the agent
# NOTE: If you use wrappers with your environment that modify rewards,
#       this will be reflected here. To evaluate with original rewards,
#       wrap environment in a "Monitor" wrapper before other wrappers.
mean_reward, std_reward = evaluate_policy(model, model.get_env(),deterministic=True,render=True, n_eval_episodes=10)
print(mean_reward,std_reward)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
146.2599667 101.88847134290344


In [16]:
episode = 10
# Enjoy trained agent
obs = env.reset()
for i in range(episode):
    dones = False
    rewards = 0
    while not dones:
        action, _states = model.predict(obs, deterministic=True)
        obs, reward, dones, info = env.step(action)
        env.render()
        rewards += reward
    print(rewards)

92.17756477768047
1.2028210934166144
-0.7710245432204264
-0.6132891415311608
1.1896247725657727
-0.8689089466011619
-0.8515214188520399
1.3546029603247807
-0.906917532456698
1.161606216906873
